In [37]:
import torch
import numpy as np

In [38]:
from config import Config
from data import Data
from channel import Channel

In [246]:
config = Config(N_transmit_antenna=128, N_active_antenna=10, N_receive_antenna=32, block_length=5, channel_length=2, batch=1, generator_mode='random', channel_truncation='tail')
data = Data(config)
channel = Channel(config)

In [252]:
W = np.zeros((config.Lout, config.Lin))
for l in np.arange(config.Lh):
    W += np.eye(config.Lout, config.Lin, -l) 
W

array([[1., 0., 0., 0., 0.],
       [1., 1., 0., 0., 0.],
       [0., 1., 1., 0., 0.],
       [0., 0., 1., 1., 0.],
       [0., 0., 0., 1., 1.],
       [0., 0., 0., 0., 1.]])

In [253]:
from scipy.linalg import toeplitz

In [254]:
c = np.zeros(config.Lin)
c[:config.Lh] = 1
c

array([1., 1., 0., 0., 0.])

In [ ]:
toeplitz(c, )